In [1]:
%matplotlib inline
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

import importlib
import os
import sys
import datetime

import keras.models as KM
import keras.layers as KL
from keras.optimizers import Adam
from keras.utils import multi_gpu_model

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import util

cur_dir = os.path.abspath('.')
root_dir = os.path.dirname(cur_dir)
sys.path.append(os.path.join(cur_dir, '3DUnetCNN'))

import unet3d.my_generator
import unet3d.my_augment
from unet3d.model import unet_model_3d
from unet3d.training import load_old_model, train_model
from unet3d.metrics import dice_coefficient_loss, get_label_dice_coefficient_function, dice_coefficient

Using TensorFlow backend.
/home/zhangsiyu/siyu/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
now = datetime.datetime.utcnow() + datetime.timedelta(hours=8)
config = dict()
config["pool_size"] = (2, 2, 2)  # pool size for the max pooling operations
# config["image_shape"] = (144, 144, 144)  # This determines what shape the images will be cropped/resampled to.
config["patch_shape"] = (128, 128, 128)  # switch to None to train on the whole image
config["labels"] = (1, 5, 31, 32)  # the label numbers on the input image
config["n_labels"] = len(config["labels"])
# config["all_modalities"] = ["t1", "t1ce", "flair", "t2"]
# config["training_modalities"] = config["all_modalities"]  # change this if you want to only use some of the modalities
# config["nb_channels"] = len(config["training_modalities"])
config["nb_channels"] = 1
if "patch_shape" in config and config["patch_shape"] is not None:
    config["input_shape"] = tuple([config["nb_channels"]] + list(config["patch_shape"]))
else:
    config["input_shape"] = tuple([config["nb_channels"]] + list(config["image_shape"]))
config["truth_channel"] = config["nb_channels"]
config["deconvolution"] = True  # if False, will use upsampling instead of deconvolution

config["batch_size"] = 1
config["validation_batch_size"] = 1
config["n_epochs"] = 300  # cutoff the training after this many epochs
config["patience"] = 10  # learning rate will be reduced after this many epochs if the validation loss is not improving
config["early_stop"] = 50  # training will be stopped after this many epochs without the validation loss improving
config["initial_learning_rate"] = 0.00005
config["learning_rate_drop"] = 0.5  # factor by which the learning rate will be reduced
config["validation_split"] = 0.1  # portion of the data that will be used for training
config["flip"] = True  # augments the data by randomly flipping an axis during
config["permute"] = False  # data shape must be a cube. Augments the data by permuting in various directions
config["distort"] = None  # switch to None if you want no distortion
config["augment"] = config["flip"] or config["distort"]
config["validation_patch_overlap"] = 0  # if > 0, during training, validation patches will be overlapping
config["training_patch_start_offset"] = (40, 30, 30)  # randomly offset the first patch index by up to this offset
config["skip_blank"] = True  # if True, then patches without any target will be skipped

config["data_path"] = os.path.abspath("./data/3d/")
config["model_file"] = os.path.abspath("./model_unet3d_{}.h5".format(now.strftime('%Y%m%d_%H%M%S')))
# config["model_file"] = os.path.abspath("./pre-trained/tumor_segmentation_model.h5")
config["training_file"] = os.path.abspath("training_uids.pkl")
config["validation_file"] = os.path.abspath("validation_uids.pkl")
config["overwrite"] = True  # If True, will previous files. If False, will use previously written files.

In [3]:
importlib.reload(unet3d.my_generator)
importlib.reload(unet3d.my_augment)

# get training and testing generators
train_generator, validation_generator, n_train_steps, n_validation_steps = unet3d.my_generator.get_training_and_validation_generators(
    config['data_path'],
    batch_size=config["batch_size"],
    validation_split=config["validation_split"],
    overwrite=config['overwrite'],
    validation_keys_file=config["validation_file"],
    training_keys_file=config["training_file"],
    n_labels=config["n_labels"],
    labels=config["labels"],
    patch_shape=config["patch_shape"],
    validation_batch_size=config["validation_batch_size"],
    validation_patch_overlap=config["validation_patch_overlap"],
    training_patch_start_offset=config["training_patch_start_offset"],
    permute=config["permute"],
    augment=config["augment"],
    skip_blank=config["skip_blank"],
    augment_flip=config["flip"],
    augment_distortion_factor=config["distort"],
#     calculate_steps=False,
    )
# n_train_steps, n_validation_steps = 1200, 120

spliting training and validation data ...
Creating validation split...
build training generator ...
build validation generator ...
calculating training steps ...
counting number of patches from 1293 samples ...
Number of training steps:  6812
calculating validation steps ...
counting number of patches from 143 samples ...
Number of validation steps:  739


In [4]:
# import pickle
# with open(config["training_file"], 'rb') as f:
#     training_list = pickle.load(f)
# with open(config["validation_file"], 'rb') as f:
#     validation_list = pickle.load(f)
# print(training_list)
# print(validation_list)

In [5]:
# data = next(train_generator)
# x, y = data
# print(x.shape)
# print(y.shape)

# util.plot_ct_as_slide(x[0][0], start=0, end=None, interval=50)

In [6]:
# util.plot_ct_as_slide(y[0][0], start=0, end=None, interval=50)

In [ ]:
# if not config['overwrite'] and os.path.exists(config["model_file"]):
model = load_old_model('./pre-trained/tumor_segmentation_model.h5')
for layer in model.layers:
    layer.trainable = False
#     if layer.name.startswith('new_') or layer.name.startswith('append_'):
#         layer.trainable = True
#     else:
#         layer.trainable = False
    
# else:
# # instantiate new model
# model = unet_model_3d(
#     input_shape=config["input_shape"],
#     pool_size=config["pool_size"],
#     n_labels=config["n_labels"],
#     initial_learning_rate=config["initial_learning_rate"],
#     deconvolution=config["deconvolution"])

In [8]:
# model.summary(line_length=120)

# # from keras.utils import plot_model
# import keras.utils
# importlib.reload(keras.utils)
# keras.utils.plot_model(model, to_file='_model.png', show_shapes=True)

In [9]:
input_shape = (1,) + tuple(config['patch_shape'])
new_input = KL.Input(input_shape, name='new_input_1')
new_conv1 = KL.Conv3D(4, (3, 3, 3), padding='same', activation='relu', name='new_conv3d_1')
new_conv2 = KL.Conv3D(4, (3, 3, 3), padding='same', activation='relu', name='new_conv3d_2')
new_maxpool1 = KL.MaxPool3D(name='new_maxpooling3d_1')
input_ = new_conv1(new_input)
input_ = new_conv2(input_)
input_ = new_maxpool1(input_)

mid_model = KM.Model(inputs=model.input, outputs=model.layers[-3].output)
out = mid_model(input_)

append_upsample1 = KL.Conv3DTranspose(16, (3, 3, 3), strides=(2, 2, 2), padding='same', activation='relu', name='append_upsampling_1')
append_conv1 = KL.Conv3D(8, (3, 3, 3), padding='same', activation='relu', name='append_conv3d_1')
append_conv2 = KL.Conv3D(4, (3, 3, 3), padding='same', activation='relu', name='append_conv3d_2')
out = append_upsample1(out)
out = append_conv1(out)
out = append_conv2(out)

new_model = KM.Model(new_input, out)
new_model.summary(line_length=120)

________________________________________________________________________________________________________________________
Layer (type)                                          Output Shape                                    Param #           
new_input_1 (InputLayer)                              (None, 1, 128, 128, 128)                        0                 
________________________________________________________________________________________________________________________
new_conv3d_1 (Conv3D)                                 (None, 4, 128, 128, 128)                        112               
________________________________________________________________________________________________________________________
new_conv3d_2 (Conv3D)                                 (None, 4, 128, 128, 128)                        436               
________________________________________________________________________________________________________________________
new_maxpooling3d_1 (MaxPooling3D

In [10]:
mid_model.summary(line_length=120)

________________________________________________________________________________________________________________________
Layer (type)                           Output Shape               Param #       Connected to                            
input_1 (InputLayer)                   (None, 4, 64, 64, 64)      0                                                     
________________________________________________________________________________________________________________________
conv3d_1 (Conv3D)                      (None, 32, 64, 64, 64)     3488          input_1[0][0]                           
________________________________________________________________________________________________________________________
activation_1 (Activation)              (None, 32, 64, 64, 64)     0             conv3d_1[0][0]                          
________________________________________________________________________________________________________________________
conv3d_2 (Conv3D)               

In [12]:
# new_model = multi_gpu_model(new_model, gpus=2)
new_model.compile(optimizer=Adam(lr=config['initial_learning_rate']), loss=dice_coefficient_loss, metrics=[dice_coefficient])

In [11]:
# new_model = load_old_model('./model_unet3d_20190721_202302.h5')
# new_model = multi_gpu_model(new_model, gpus=2)
# new_model.summary(line_length=120)

Loading pre-trained model
________________________________________________________________________________________________________________________
Layer (type)                                          Output Shape                                    Param #           
new_input_1 (InputLayer)                              (None, 1, 128, 128, 128)                        0                 
________________________________________________________________________________________________________________________
new_conv3d_1 (Conv3D)                                 (None, 4, 128, 128, 128)                        112               
________________________________________________________________________________________________________________________
new_conv3d_2 (Conv3D)                                 (None, 4, 128, 128, 128)                        436               
________________________________________________________________________________________________________________________
new_ma

In [ ]:
# run training
train_model(
    model=new_model,
    model_file=config["model_file"],
    training_generator=train_generator,
    validation_generator=validation_generator,
    steps_per_epoch=n_train_steps,
    validation_steps=n_validation_steps,
    initial_learning_rate=config["initial_learning_rate"],
    learning_rate_drop=config["learning_rate_drop"],
    learning_rate_patience=config["patience"],
    early_stopping_patience=config["early_stop"],
    n_epochs=config["n_epochs"])

starting from learning rate 5e-05 ...
Epoch 1/300
6812/6812 [==============================] - 10938s 2s/step - loss: -0.0306 - dice_coefficient: 0.0306 - val_loss: -0.0467 - val_dice_coefficient: 0.0467
Epoch 2/300
6812/6812 [==============================] - 11044s 2s/step - loss: -0.0442 - dice_coefficient: 0.0442 - val_loss: -0.0589 - val_dice_coefficient: 0.0589
Epoch 3/300
3086/6812 [============>.................] - ETA: 1:33:25 - loss: -0.0551 - dice_coefficient: 0.0551

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6812/6812 [==============================] - 10767s 2s/step - loss: -0.0536 - dice_coefficient: 0.0536 - val_loss: -0.0672 - val_dice_coefficient: 0.0672
Epoch 4/300
6812/6812 [==============================] - 10474s 2s/step - loss: -0.0597 - dice_coefficient: 0.0597 - val_loss: -0.0716 - val_dice_coefficient: 0.0716
Epoch 5/300
6812/6812 [==============================] - 10270s 2s/step - loss: -0.0631 - dice_coefficient: 0.0631 - val_loss: -0.0740 - val_dice_coefficient: 0.0740
Epoch 6/300
6812/6812 [==============================] - 10208s 1s/step - loss: -0.0709 - dice_coefficient: 0.0709 - val_loss: -0.0784 - val_dice_coefficient: 0.0784
Epoch 7/300
6812/6812 [==============================] - 10214s 1s/step - loss: -0.0744 - dice_coefficient: 0.0744 - val_loss: -0.0834 - val_dice_coefficient: 0.0834
Epoch 8/300
6812/6812 [==============================] - 10211s 1s/step - loss: -0.0763 - dice_coefficient: 0.0763 - val_loss: -0.0804 - val_dice_coefficient: 0.0804
Epoch 9/300
1469